
Estimate average equivalent residential unit based on impervious surface GIS data intersected to residential land use and residential zoning codes:
    

In [ ]:
#union imput features including impervious area polygons, lawrence zoning polygons, land use 2023 to sql server for sql exploration
arcpy.analysis.Union(
    in_features=r"'Land Use 2023 YB' #;'Lawrence Zoning' #;'SWU Impervious Area\Impervious Area' #",
    out_feature_class=r'C:\Users\kgonterwitz\OneDrive - City of Lawrence KS\Documents\ArcGIS\Projects\KyleProj2023\testgdb (2).sde\"LAWRENCE\KGONTERWITZ".Union_of_LandUseZoningImpervious',
    join_attributes="ALL",
    cluster_tolerance=None,
    gaps="GAPS"
)

In [ ]:
#this dissolve sums impervious surface area to the parcel or land use geometry based on parcel ID definition
#it does not include ROW, voids in the PARCEL layer are not included in the parcel shape union
#this mostly matches the data in the impervious area layers

arcpy.management.Dissolve(
    in_features=r'"LAWRENCE\KGONTERWITZ".Union_of_LandUseZoningImpervious',
    out_feature_class=r"C:\Users\kgonterwitz\OneDrive - City of Lawrence KS\Documents\ArcGIS\Projects\KyleProj2023\KyleProjDefault.gdb\LAWRENCEKGONTERWITZ_Dissolve",
    dissolve_field="JOINPIN",
    statistics_fields="SYSCALACRES MEAN;Res_Yearbuilt FIRST;impArea SUM;flivunits MEDIAN;Shape.STArea() SUM;Imp_ID COUNT",
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

In [ ]:
#dissolve to the land use parcel ID and common land use generalization
arcpy.management.Dissolve(
    in_features=r'"LAWRENCE\KGONTERWITZ".Union_of_LandUseZoningImpervious',
    out_feature_class=r"C:\Users\kgonterwitz\OneDrive - City of Lawrence KS\Documents\ArcGIS\Projects\KyleProj2023\KyleProjDefault.gdb\LAWRENCEKGONTERWITZ_Dissolve",
    dissolve_field="JOINPIN;Generalization",
    statistics_fields="SYSCALACRES MEAN;Res_Yearbuilt FIRST;impArea SUM;flivunits MEDIAN;Shape.STArea() SUM;Imp_ID COUNT",
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

In [ ]:
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="LAWRENCEKGONTERWITZ_Dissolve",
    selection_type="NEW_SELECTION",
    where_clause="Generalization = 'Single Family Residential' And MEDIAN_flivunits > 0",
    invert_where_clause=None
)

In [1]:

#I dissolved the dissolved layer after querying it Generalization = 'Single Family Residential'
#use Generalization = 'Single Family Residential' And MEDIAN_flivunits > 0

arcpy.management.Dissolve(
    in_features="LAWRENCEKGONTERWITZ_Dissolve_defq",
    out_feature_class=r"C:\Users\kgonterwitz\OneDrive - City of Lawrence KS\Documents\ArcGIS\Projects\KyleProj2023\KyleProjDefault.gdb\LAWRENCEKGONTERWITZ_Dissolve_ERU",
    dissolve_field="Generalization",
    statistics_fields="SUM_impArea SUM;Shape_Area SUM;JOINPIN COUNT",
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

<Result 'C:\\Users\\kgonterwitz\\OneDrive - City of Lawrence KS\\Documents\\ArcGIS\\Projects\\KyleProj2023\\KyleProjDefault.gdb\\LAWRENCEKGONTERWITZ_Dissolve_ERU'>

In [2]:
#calculate the ERU as the sum of the SF res impervious area divided by the number of land use parcel ID units
arcpy.management.CalculateField(
    in_table="LAWRENCEKGONTERWITZ_Dissolve_ERU",
    field="ERU_AVG_IMP",
    expression="!SUM_SUM_impArea!/!COUNT_JOINPIN!",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'LAWRENCEKGONTERWITZ_Dissolve_ERU'>